# AIDA Nanopublication Generator

Creates AIDA (Atomic, Independent, Declarative, Absolute) sentence nanopublications from a JSON configuration file.

**Template:** [AIDA Sentence Template](https://w3id.org/np/RALmXhDw3rHcMveTgbv8VtWxijUHwnSqhCmtJFIPKWVaA)

## AIDA Sentences
- **A**tomic: A sentence describing one thought that cannot be further broken down
- **I**ndependent: A sentence that can stand on its own, without external references
- **D**eclarative: A complete sentence ending with a full stop that could be true or false
- **A**bsolute: A sentence describing the core of a claim ignoring uncertainty

In [ ]:
import json
import sys
import urllib.parse
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

from nanopub_utils import (
    NanopubGenerator, load_config, save_nanopub,
    make_literal, PREFIXES
)

In [ ]:
# AIDA-specific constants
HYCL_AIDA_SENTENCE = "http://purl.org/petapico/o/hycl#AIDA-Sentence"
HYCL_NS = "http://purl.org/petapico/o/hycl"
CITO_OBTAINS_SUPPORT = "http://purl.org/spar/cito/obtainsSupportFrom"
SCHEMA_ABOUT = "http://schema.org/about"
SKOS_RELATED = "http://www.w3.org/2004/02/skos/core#related"
AIDA_NS = "http://purl.org/aida/"

class AidaNanopubGenerator(NanopubGenerator):
    """Generator for AIDA sentence nanopublications matching Nanodash format."""
    
    def __init__(self, config: dict, nanopub_config: dict):
        # Merge metadata with individual nanopub config
        merged_config = {
            **config.get('metadata', {}),
            **nanopub_config,
            'template_uri': config.get('template_uri')
        }
        super().__init__(merged_config)
        
        # Add nanopub types for AIDA
        self.add_nanopub_type(HYCL_AIDA_SENTENCE)
        self.add_nanopub_type(HYCL_NS)
    
    def generate_assertion(self) -> str:
        """Generate the AIDA assertion graph using full URIs."""
        aida_sentence = self.config['aida_sentence']
        topics = self.config.get('topics', [])
        publication = self.config.get('related_publication')
        dataset = self.config.get('related_dataset')
        project = self.config.get('related_project')
        
        # Create URL-encoded AIDA sentence URI
        aida_uri = AIDA_NS + urllib.parse.quote(aida_sentence, safe='')
        
        # Set this as the introduced URI
        self.set_introduces(aida_uri)
        
        # Truncate label if needed (like Nanodash does)
        label_text = aida_sentence
        if len(label_text) > 100:
            label_text = label_text[:100] + "..."
        self.config['label'] = f"AIDA sentence: {label_text}"
        
        lines = [f"{self.sub_prefix}:assertion {{"]
        
        # Build the assertion using full URIs (not prefixed)
        # Collect all predicates for the AIDA sentence
        predicates = []
        predicates.append(f"    a <{HYCL_AIDA_SENTENCE}>")
        
        # Add related publication(s)
        if publication:
            pub_uri = publication if publication.startswith('http') else f"https://doi.org/{publication}"
            predicates.append(f"    <{CITO_OBTAINS_SUPPORT}> <{pub_uri}>")
        
        # Add related dataset
        if dataset:
            # Add to existing cito:obtainsSupportFrom or create new
            predicates.append(f"    <{CITO_OBTAINS_SUPPORT}> <{dataset}>")
        
        # Add topic relationships
        for topic in topics:
            topic_uri = topic.get('uri', '')
            topic_label = topic.get('label', '')
            if topic_uri:
                predicates.append(f"    <{SCHEMA_ABOUT}> <{topic_uri}>")
                # Track Wikidata labels for pubinfo
                if topic_label:
                    self.add_wikidata_label(topic_uri, topic_label)
        
        # Add related project
        if project:
            predicates.append(f"    <{SKOS_RELATED}> <{project}>")
        
        # Format with proper semicolons
        lines.append(f"  <{aida_uri}>")
        for i, pred in enumerate(predicates):
            if i < len(predicates) - 1:
                lines.append(f"{pred};")
            else:
                lines.append(f"{pred} .")
        
        lines.append("}")
        return "\n".join(lines)

In [ ]:
# Configuration
CONFIG_FILE = "../config/vbae208_aida.json"  # Change this to use different config
OUTPUT_DIR = "../output/aida"

# Create output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

In [ ]:
# Load configuration
config = load_config(CONFIG_FILE)

print(f"Source paper: {config['metadata']['source_paper']['title']}")
print(f"DOI: {config['metadata']['source_paper']['doi']}")
print(f"Number of AIDA nanopublications to generate: {len(config['nanopublications'])}")
print()

for i, np_config in enumerate(config['nanopublications'], 1):
    print(f"{i}. {np_config['aida_sentence'][:80]}...")

In [ ]:
# Generate nanopublications
generated_files = []

for np_config in config['nanopublications']:
    # Create generator
    generator = AidaNanopubGenerator(config, np_config)
    
    # Generate nanopub content
    nanopub_content = generator.generate()
    
    # Save to file
    output_file = f"{OUTPUT_DIR}/{np_config['id']}.trig"
    save_nanopub(nanopub_content, output_file)
    generated_files.append(output_file)
    
    print(f"Generated: {output_file}")

print(f"\nTotal generated: {len(generated_files)} nanopublications")

In [ ]:
# Preview first generated nanopublication
if generated_files:
    print(f"Preview of {generated_files[0]}:\n")
    print("=" * 80)
    with open(generated_files[0], 'r') as f:
        print(f.read())

## Next Steps

1. Review the generated `.trig` files in the output directory
2. Use **sign_and_publish_nanopub.ipynb** to sign and publish
3. To use with a different paper, create a new JSON config file and update `CONFIG_FILE`